In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import callbacks
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

import keras
from keras import *
from keras.layers import *
from pythainlp import word_tokenize
from pythainlp.word_vector import *

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('Suicidal_K5_Train.csv').drop(columns='Unnamed: 0')

In [ ]:
df

In [ ]:
wModel = get_model()
thai2dict = {}
for word in wModel.index_to_key:
    thai2dict[word] = wModel[word]
thai2vec = pd.DataFrame.from_dict(thai2dict,orient='index')
thVocab = thai2vec.index.to_list()

In [ ]:
thai2vec.head(10)

In [ ]:
ll = len(thai2vec)


In [ ]:
for vidx in range(ll):
    if vidx % 100 == 0:
        print('\r' + str(vidx),end='')
    aa = thai2vec.iloc[[vidx]]
    ab = aa.values.tolist()
    if vidx == 0:
        vect = ab
    else:
        vect = np.vstack((vect,ab))

print("\n", vect.shape)

In [ ]:
def convert2num(convData):
    for ci in range(len(convData)):
        if convData[ci] == "Level 1":
            convData[ci] = 0
        elif convData[ci] == "Level 2":
            convData[ci] = 1
        elif convData[ci] == "Level 3":
            convData[ci] = 2
        elif convData[ci] == "Level 4":
            convData[ci] = 3
        elif convData[ci] == "Level 5":
            convData[ci] = 4
        elif convData[ci] == "Other":
            convData[ci] = 5
    return convData

In [ ]:
def tokenWord(wordTarget):
    wordToken = word_tokenize(wordTarget, engine='attacut')
    return wordToken

In [ ]:
def convWord(cw):
    cWord = cw
    for ti in range(len(cWord)):
        if cWord[ti] == ' ':
            cWord[ti] = ''
        elif cWord[ti] not in thVocab:
            cWord[ti] = ''
    return cWord

In [ ]:
def token2index(t2idx):
    w2index = []
    for wi in range(len(t2idx)):
        if t2idx[wi] in thVocab:
            w2index.append(thVocab.index(t2idx[wi]))
    return np.array(w2index)

In [ ]:
def findMaxArray(fma):
    maxlen = 0
    for mi in range(len(fma)):
        nA = len(fma[mi])
        if nA > maxlen:
            maxlen = nA
    return maxlen

In [ ]:
def fill0in(f0i):
    fMax = findMaxArray(f0i)
    for fi, ax in enumerate(f0i):
        if len(ax) < fMax:
            f0i[fi] = np.hstack((ax , np.zeros(fMax-len(ax))))
        f0i1 = np.array(f0i)
    return f0i1

In [ ]:
def prepare2train(ipt):
    pre2t = []
    for pidx in range(len(ipt)):
        wp1 = ipt[pidx]
        pre2t.append(token2index(convWord(tokenWord(wp1))))
    return pre2t

In [ ]:
tweet = df['Tweet']
tweetArray = tweet.to_list()
label = df['Label (Specialist)']

In [ ]:
Xtrain = prepare2train(tweetArray)
Ytrain = convert2num(label.to_list())

In [ ]:
YTrainIdx = np.array(Ytrain)
XTrainIdx = fill0in(Xtrain)

In [ ]:
XTrainIdx.shape

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vect.shape[0],vect.shape[1],name='embed'))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.get_layer('embed').set_weights([vect])
model.get_layer('embed').trainable = True

In [ ]:
class myCallback(callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      self.model.stop_training = True
      print("\nCallback Accuracy มากกว่า 95%!")
callbacks = myCallback()

In [ ]:
model.fit(XTrainIdx,YTrainIdx,epochs=50,batch_size=32,callbacks=[callbacks])

In [ ]:
test_df = pd.read_csv('Suicidal_K5_Test.csv').drop(columns='Unnamed: 0')
test_label = test_df['Label (Specialist)']
X_test = test_df['Tweet']

test_array = X_test.to_list()

X_test = prepare2train(test_array)
y_test = convert2num(test_label)

In [ ]:
YTestIdx = np.array(y_test)
XTestIdx = fill0in(X_test)

In [ ]:
y_pred = np.argmax(model.predict(XTestIdx),axis=1)

In [ ]:
y_true = test_df['Label (Specialist)']
y_true

In [ ]:
XTestIdx = np.asarray(XTestIdx).astype(np.float32)
YTestIdx = np.asarray(YTestIdx).astype(np.float32)

In [ ]:
accr = model.evaluate(XTestIdx,YTestIdx)

In [ ]:
y_true = y_true.tolist()
y_pred = y_pred.tolist()

In [ ]:
print(classification_report(y_true, y_pred))